In [24]:
# Imports
import pandas as pd
import warnings
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# Filter warnings
warnings.simplefilter("ignore")

In [15]:
# Loading CSV files
salesData = pd.read_csv('../Quiz4/Quiz4-data/sales_data.csv')
customerData = pd.read_csv('../Quiz4/Quiz4-data/customer_data.csv')

In [ ]:
# -------------QUESTION 1---------------

In [17]:
# (Q1A) Data Statistics
## First row of data is also printed
salesRows, salesColumns = salesData.shape
customerRows, customerColumns = customerData.shape
salesDomains = salesData.iloc[0]
customerDomains = customerData.iloc[0]

print("Domains of Sales Data:")
print(salesDomains)
print("\n")
print("Domains of Customer Data:")
print(customerDomains)
print("\n")
print(f"Number of Sales Rows: {salesRows}")
print(f"Number of Sales Features: {salesColumns}")
print("\n")
print(f"Number of Customer Rows: {customerRows}")
print(f"Number of Customer Features: {customerColumns}")

Domains of Sales Data:
invoice_no          I138884
customer_id         C241288
category           Clothing
quantity                  5
price                1500.4
invoice_date     05-08-2022
shopping_mall        Kanyon
Name: 0, dtype: object


Domains of Customer Data:
customer_id           C241288
gender                 Female
age                      28.0
payment_method    Credit Card
Name: 0, dtype: object


Number of Sales Rows: 99457
Number of Sales Features: 7


Number of Customer Rows: 99457
Number of Customer Features: 4


In [18]:
# (Q1B) 1. Missing data
salesMissingData = salesData.isnull().sum()
customerMissingData = customerData.isnull().sum()

missing_data = pd.DataFrame({
    'Sales Missing Data': salesMissingData,
    'Customers Missing Data': customerMissingData
})

print("Sales Missing Values: ")
print(salesMissingData)
print("\n")
print("Customers Missing Values: ")
print(customerMissingData)
print("\n")

Sales Missing Values: 
invoice_no       0
customer_id      0
category         0
quantity         0
price            0
invoice_date     0
shopping_mall    0
dtype: int64


Customers Missing Values: 
customer_id         0
gender              0
age               119
payment_method      0
dtype: int64




In [19]:
# 2. Unique Products and the product with the most bought
category = 'category'
unique_items_count = salesData[category].value_counts()
largest_items = unique_items_count.idxmax()

print(f"Number of Unique Items: {len(unique_items_count)}")
print(f"Most common item is '{largest_items}' with {unique_items_count[largest_items]} items bought")

Number of Unique Items: 8
Most common item is 'Clothing' with 34487 items bought


In [20]:
# 3. Who buys more
gender = 'gender'
unique_items_count = customerData[gender].value_counts()
largest_gender = unique_items_count.idxmax()

print(f"Gender that buys more: {largest_gender}")

# Top 3 things each gender buys
merge_data = pd.merge(salesData, customerData, on='customer_id')
group_data = merge_data.groupby(['gender', 'category']).size().reset_index(name='Amount Bought')
top_categories = group_data.groupby('gender').apply(lambda x: x.nlargest(3, 'Amount Bought')).reset_index(drop=True)
print(top_categories)

Gender that buys more: Female
   gender         category  Amount Bought
0  Female         Clothing          20652
1  Female        Cosmetics           9070
2  Female  Food & Beverage           8804
3    Male         Clothing          13835
4    Male        Cosmetics           6027
5    Male  Food & Beverage           5972


In [21]:
# 4. Is cash or credit used more
payment_method = 'payment_method'
unique_items_count = customerData[payment_method].value_counts()
payment = unique_items_count.idxmax()

print(f"Payment Method used the most: {payment}")

Payment Method used the most: Cash


In [22]:
# 5. Month that sees the most purchases
## Output is 1 which indicates January is the month with the most purchases
salesData['invoice_date'] = pd.to_datetime(salesData['invoice_date'], format='%d-%m-%Y')
salesData['month'] = salesData['invoice_date'].dt.month
commonMonth = salesData['month'].mode().values[0]
print(f"Month with the most purchases: {commonMonth}")


Month with the most purchases: 1


In [23]:
# 6. Dividing purchases into bins of $500
bin_limits = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, float('inf')]
bin_labels = ['$0-$500', '$501-$1000', '$1001-$1500', '$1501-$2000', '$2001-$2500', '$2501-3000', '3001-3500', '3501-4000','>$4000']
salesData['price_bin'] = pd.cut(salesData['price'], bins=bin_limits, labels=bin_labels, right=False)

# Bin with the highest purchase
highestPurchase = salesData.groupby('price_bin')['price'].sum().idxmax()

# Bin with most purchases
mostPurchases = salesData['price_bin'].value_counts().idxmax()

print(f"Bin with highest purchase amount is: {highestPurchase}")
print(f"Bin with most purchases is: {mostPurchases}")

# Bin with most purchases in the month with most purchases
mostPurchasesMonth = salesData[salesData['month'] == commonMonth]
mostPurchasesBin = mostPurchasesMonth['price_bin'].value_counts().idxmax()

print(f"Bin with most purchases in month with the most purchases: {mostPurchasesBin}")

# ------------------------------------------------------------
#  Data shows that the bin with the most purchases in the month
#  with the most purchases is the same as the overall bin with
#  the most purchases ($0-$500)
# ------------------------------------------------------------




Bin with highest purchase amount is: $1501-$2000
Bin with most purchases is: $0-$500
Bin with most purchases in month with the most purchases: $0-$500


In [ ]:
# -------------QUESTION 2----------------